In [1]:
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
np.set_printoptions(suppress=True)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
validationData = validationD.reshape((validationD.shape[0], validationD.shape[1], 1))
validationLabel = np_utils.to_categorical(validationL, 4)
testData = testD.reshape((testD.shape[0], testD.shape[1], 1))
testLabel = np_utils.to_categorical(testL, 4)

print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)
print('Vali Data: ', validationData.shape)
print('Vali Label: ', validationLabel.shape)
print('Test Data: ', testData.shape)
print('Test Label: ', testLabel.shape)

Train Data: (5078, 9000, 1)
Train Label:  (5078, 4)
Vali Data:  (2032, 9000, 1)
Vali Label:  (2032, 4)
Test Data:  (3041, 9000, 1)
Test Label:  (3041, 4)


In [3]:
graph = tf.Graph()

with graph.as_default():
    inputs = tf.placeholder(dtype=tf.float32, shape=(None, 9000, 1), name='inputs')
    labels = tf.placeholder(dtype=tf.float32, shape=(None, 4), name='labels')    

    conv1 = tf.layers.conv1d(inputs, filters=32, kernel_size=7, padding='valid', activation=tf.nn.relu, name='conv1d_1')
    max_pool_1 = tf.layers.max_pooling1d(conv1, pool_size=2, strides=2)
    
    conv2 = tf.layers.conv1d(max_pool_1, filters=32, kernel_size=7, padding='valid', activation=tf.nn.relu, name='conv1d_2')
    max_pool_2 = tf.layers.max_pooling1d(conv2, pool_size=2, strides=2)
    
    conv3 = tf.layers.conv1d(max_pool_2, filters=64, kernel_size=7, padding='valid', activation=tf.nn.relu, name='conv1d_3')
    max_pool_3 = tf.layers.max_pooling1d(conv3, pool_size=2, strides=2)
    
    conv4 = tf.layers.conv1d(max_pool_3, filters=64, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_4')
    max_pool_4 = tf.layers.max_pooling1d(conv4, pool_size=2, strides=2)
    
    conv5 = tf.layers.conv1d(max_pool_4, filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_5')
    max_pool_5 = tf.layers.max_pooling1d(conv5, pool_size=2, strides=2)
    
    conv6 = tf.layers.conv1d(max_pool_5, filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_6')
    max_pool_6 = tf.layers.max_pooling1d(conv6, pool_size=2, strides=2)
    dropout_1 = tf.layers.dropout(max_pool_6, rate=0.5)
    
    conv7 = tf.layers.conv1d(dropout_1, filters=256, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_7')
    max_pool_7 = tf.layers.max_pooling1d(conv7, pool_size=2, strides=2)
    
    conv8 = tf.layers.conv1d(max_pool_7, filters=256, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_8')
    max_pool_8 = tf.layers.max_pooling1d(conv8, pool_size=2, strides=2)
    dropout_2 = tf.layers.dropout(max_pool_8, rate=0.5)
    
    conv9 = tf.layers.conv1d(dropout_2, filters=512, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_9')
    max_pool_9 = tf.layers.max_pooling1d(conv9, pool_size=2, strides=2)
    dropout_3 = tf.layers.dropout(max_pool_9, rate=0.5)
    
    conv10 = tf.layers.conv1d(dropout_3, filters=512, kernel_size=3, padding='valid', activation=tf.nn.relu, name='conv1d_10')
    
    flatten = tf.contrib.layers.flatten(conv10)
    dense_1 = tf.layers.dense(inputs=flatten, units=128, activation=tf.nn.relu)
    dropout_4 = tf.layers.dropout(dense_1, rate=0.5)
    dense_2 = tf.layers.dense(inputs=dropout_4, units=32, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=dense_2, units=4)
    output_1 = tf.contrib.layers.softmax(output)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=labels))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
 
    # Accuracy
    correct_pred = tf.equal(tf.argmax(output_1, 1), tf.argmax(labels, 1))
    cast_ = tf.cast(correct_pred, tf.float32)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [4]:
iteration = 100
batch_size = 70

train_length = trainData.shape[0]
start = 0
end = batch_size

def batch_generation(data_length):
    row = data_length//batch_size
    col = batch_size
    arr = np.arange(row*col)
    np.random.shuffle(arr)
    arr = arr.reshape((row, col))
    return arr

batch_index = batch_generation(trainData.shape[0])

In [5]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    tensor_name_list = [tensor.name for tensor in tf.get_default_graph().as_graph_def().node]
    for tensor_name in tensor_name_list:
        print(tensor_name,'\n')

#     bar = tf.get_default_graph().get_tensor_by_name('Flatten/flatten/Reshape:0')
#     print(bar)
    

inputs 

labels 

conv1d_1/kernel/Initializer/random_uniform/shape 

conv1d_1/kernel/Initializer/random_uniform/min 

conv1d_1/kernel/Initializer/random_uniform/max 

conv1d_1/kernel/Initializer/random_uniform/RandomUniform 

conv1d_1/kernel/Initializer/random_uniform/sub 

conv1d_1/kernel/Initializer/random_uniform/mul 

conv1d_1/kernel/Initializer/random_uniform 

conv1d_1/kernel 

conv1d_1/kernel/Assign 

conv1d_1/kernel/read 

conv1d_1/bias/Initializer/zeros 

conv1d_1/bias 

conv1d_1/bias/Assign 

conv1d_1/bias/read 

conv1d_1/dilation_rate 

conv1d_1/conv1d/ExpandDims/dim 

conv1d_1/conv1d/ExpandDims 

conv1d_1/conv1d/ExpandDims_1/dim 

conv1d_1/conv1d/ExpandDims_1 

conv1d_1/conv1d/Conv2D 

conv1d_1/conv1d/Squeeze 

conv1d_1/BiasAdd 

conv1d_1/Relu 

max_pooling1d/ExpandDims/dim 

max_pooling1d/ExpandDims 

max_pooling1d/MaxPool 

max_pooling1d/Squeeze 

conv1d_2/kernel/Initializer/random_uniform/shape 

conv1d_2/kernel/Initializer/random_uniform/min 

conv1d_2/kernel/Initialize

In [6]:
with tf.Session(graph=graph) as sess:

    bar = tf.get_default_graph().get_tensor_by_name('conv1d_1/Relu:0')
    print(bar)

Tensor("conv1d_1/Relu:0", shape=(?, 8994, 32), dtype=float32)


In [7]:
def summary():
    layer_name = 'conv1d_'
    layers = []
    counter = 1
    for tensor in tf.get_default_graph().as_graph_def().node:
        if tensor.name == layer_name+str(counter)+'/Relu':
            layers.append(tensor.name)
            counter += 1
        if tensor.name == "Flatten/flatten/Reshape":
            layers.append(tensor.name)
    for tensor_name in layers:
        print(tensor_name, tf.get_default_graph().get_tensor_by_name(tensor_name+':0').shape)

In [8]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    summary()

    batch_index = batch_generation(trainData.shape[0])
    for epoch in range(iteration):
        for index in batch_index:
            loss, opt, acc = sess.run([cost, optimizer, accuracy],
                             feed_dict={inputs: trainData[index],
                                        labels: trainLabel[index]
                                    })   
        
        val_loss, val_acc = sess.run([cost, accuracy],
                             feed_dict={inputs: validationData,
                                        labels: validationLabel
                                    })
#         print(cp)
#         print(ca)
        print("Epoch: {}/{}".format(epoch+1, iteration))
        print("loss:", "{:09f}".format(loss), "-", "acc:", acc, "-",
              "val_loss:", "{:09f}".format(val_loss), "-", "val_acc:", val_acc)

conv1d_1/Relu (?, 8994, 32)
conv1d_2/Relu (?, 4491, 32)
conv1d_3/Relu (?, 2239, 64)
conv1d_4/Relu (?, 1117, 64)
conv1d_5/Relu (?, 556, 128)
conv1d_6/Relu (?, 276, 128)
conv1d_7/Relu (?, 136, 256)
conv1d_8/Relu (?, 66, 256)
conv1d_9/Relu (?, 31, 512)
conv1d_10/Relu (?, 13, 512)
Flatten/flatten/Reshape (?, 6656)


KeyboardInterrupt: 